In [1]:
import numpy as np
import pandas as pd

##### How to use this implementation of GK Algorithm

In [30]:
from gk import Sketch

epsilon = 0.001
test_sketch = Sketch(epsilon) #init sketch object

# try on random sequence of numbers
N = 100000
random_vars = np.random.randint(0, 100, N)

for v in random_vars:
    test_sketch.insert(v)

##### Get approximate quantile

In [31]:
quant = 0.5 #get median
answer = test_sketch.quantile(quant)

print(f'{test_sketch.eps}-Approximate {quant}-quantile is {answer}')

0.001-Approximate 0.5-quantile is 50


##### Check with real answer

In [32]:
real_quantile = sorted(random_vars)[int(quant*N)]
print(f'Exact {quant}-quantile is {real_quantile}')

Exact 0.5-quantile is 49


##### How much space is used for our data structure

In [33]:
print(f'Number of tuples (v, g, d) is {test_sketch.items_num()} for N={N}')

Number of tuples (v, g, d) is 3000 for N=100000


##### Make several random tests and check if answer is indeed in the interval

In [44]:
number_of_tests = 5
for i in range(number_of_tests):
    eps = 0.05
    N = 10000
    s = Sketch(eps)
    
    vars = np.random.rand((N))
    for var in vars:
        s.insert(var)
    
    q = 0.25
    answer = s.quantile(q)
    assert answer in sorted(vars)[int(q*N)-int(eps*N):int(q*N)+int(eps*N)]
    print(f'Test {i+1} passed')

Test 1 passed
Test 2 passed
Test 3 passed
Test 4 passed
Test 5 passed


##### Analyze space complexity on different epsilons and N

In [41]:
import collections
epsilons = [0.2, 0.1, 0.05, 0.01, 0.001]
N = [1000, 5000, 10000, 100000]

results = collections.defaultdict(dict)
for n in N:
    for eps in epsilons:
        s = Sketch(eps)
        for i in range(n):
            s.insert(np.random.rand())
        results[f'N={n}'][f'eps={eps}'] = s.items_num()

df = pd.DataFrame.from_dict(results, orient='index')
df

,eps=0.2,eps=0.1,eps=0.05,eps=0.01,eps=0.001
N=1000,4,8,16,83,1000
N=5000,4,8,16,81,841
N=10000,4,7,16,76,829
N=100000,5,8,14,69,749


##### We can see that practically algorithm uses much less space than theoretical worst case

In [ ]:
import matplotlib.pyplot as plt

N = [1000, 2000, 4000, 8000, 16000, 32000, 64000, 128000]
eps = 0.01
y = []
for n in N:
    s = Sketch(eps)
    vars = np.random.rand((n))
    for var in vars:
        s.insert(var)
    
    y.append(s.items_num)